In [ ]:
sc

In [ ]:
sc.version

# positions-geographiques-des-stations-du-reseau-ratp

In [55]:
// The code was removed by DSX for sharing.

1


Name: Unknown Error
Message: <console>:85: error: not found: value display
       display(dfGeo)
       ^
StackTrace: 

In [58]:
%display dfGeo

Name: Error parsing magics!
Message: Magics [display] do not exist!
StackTrace: 

# Validatations

In [22]:
// TEST Date cast
import java.text.SimpleDateFormat
val jourFormat = new SimpleDateFormat("yyyy-MM-dd")
print(jourFormat.parse("2016-01-05"))


Tue Jan 05 00:00:00 CST 2016

In [1]:
import java.sql.Date
val rdd_csv = sc.textFile("swift://sncf." + name + "/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-semestre-2015.csv")
val headerAndRows = rdd_csv.map(line => line.split(";").map(_.trim))
// filter out header (eh. just check if the first val matches the first header name)
val data = headerAndRows.filter(_(0) != header(0))
// splits to map (header/value pairs)
val maps = data.map(splits => header.zip(splits).toMap)
// filter out the user "me"
val result = maps.filter(map => map("NB_VALD") != "Moins de 5")
// print result
result.take(1).foreach(println)

val columns=result.take(1).flatMap(a=>a.keys)
val resu=result.map{value=>
      val list=value.values.toList
      (list(0),list(1),list(2),Date.valueOf(list(3)),list(4),list(5).toDouble,list(6),list(7))
      }.toDF(columns:_*)

resu.show()
resu.printSchema()


Map(LIBELLE_ARRET -> SAINT-DENIS-PORTE DE PARIS, CODE_STIF_TRNS -> 100, CODE_STIF_ARRET -> 762, JOUR -> 2016-01-05, CODE_STIF_RES -> 110, NB_VALD -> 264, ID_REFA_LDA -> 72285, CATEGORIE_TITRE -> AMETHYSTE)
+--------------------+--------------+---------------+----------+-------------+-------+-----------+---------------+
|       LIBELLE_ARRET|CODE_STIF_TRNS|CODE_STIF_ARRET|      JOUR|CODE_STIF_RES|NB_VALD|ID_REFA_LDA|CATEGORIE_TITRE|
+--------------------+--------------+---------------+----------+-------------+-------+-----------+---------------+
|SAINT-DENIS-PORTE...|           100|            762|2016-01-05|          110|  264.0|      72285|      AMETHYSTE|
|       SAINT-GEORGES|           100|            766|2016-01-05|          110|  103.0|      71366|            TST|
|       SAINT-JACQUES|           100|            768|2016-01-05|          110|  415.0|      71041|            TST|
|        SAINT-LAZARE|           100|            769|2016-01-05|          110|15406.0|      71370|      

Name: Unknown Error
Message: <console>:238: error: not found: value df3
       df3.printSchema()
       ^
StackTrace: 

In [7]:
import spark.implicits._
val ds = spark.createDataset(resultantDF)

Name: Unknown Error
Message: <console>:249: error: overloaded method value createDataset with alternatives:
  [T](data: java.util.List[T])(implicit evidence$6: org.apache.spark.sql.Encoder[T])org.apache.spark.sql.Dataset[T] <and>
  [T](data: org.apache.spark.rdd.RDD[T])(implicit evidence$5: org.apache.spark.sql.Encoder[T])org.apache.spark.sql.Dataset[T] <and>
  [T](data: Seq[T])(implicit evidence$4: org.apache.spark.sql.Encoder[T])org.apache.spark.sql.Dataset[T]
 cannot be applied to (org.apache.spark.sql.DataFrame)
       val ds = spark.createDataset(resultantDF)
                      ^
StackTrace: 

In [ ]:
// create DataFrame from RDD (Programmatically Specifying the Schema) 
val headerColumns = rdd.first().split(",").to[List] 
// extract headers [..] first
def dfSchema(columnNames: List[String]): StructType = { StructType( Seq( StructField(name = "manager_name", dataType = StringType, nullable = false),
                                                                        StructField(name = "client_name", dataType = StringType, nullable = false),
                                                                        StructField(name = "client_gender", dataType = StringType, nullable = false),
                                                                        StructField(name = "client_age", dataType = IntegerType, nullable = false),
                                                                        StructField(name = "response_time", dataType = DoubleType, nullable = false),
                                                                        StructField(name = "satisfaction_level", dataType = DoubleType, nullable = fals) ) ) } 
// create a data row 
def row(line: List[String]): Row = { Row(line(0), line(1), line(2), line(3).toInt, line(4).toDouble, line(5).toDouble) } 
// define a schema for the file 
val schema = dfSchema(headerColumns) val data = rdd .mapPartitionsWithIndex((index, element) => if (index == 0) it.drop(1) else it) 
// skip header 
.map(_.split(",").to[List]) .map(row) 
val dataFrame = spark.createDataFrame(data, schema)


In [8]:
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("JOUR", DateType, true),
    StructField("CODE_STIF_TRNS", StringType, true),
    StructField("CODE_STIF_RES", StringType, true),
    StructField("CODE_STIF_ARRET", StringType, true),
    StructField("LIBELLE_ARRET", StringType, true),
    StructField("ID_REFA_LDA", StringType, true),
    StructField("CATEGORIE_TITRE", StringType, true),
    StructField("NB_VALD", IntegerType, true)))

val dfValidation = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    schema(customSchema).
    option("delimiter",";").
    option("nullValue", "null").
    load("swift://sncf." + name + "/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-semestre-2015.csv")
dfValidation.show(5)


+----------+--------------+-------------+---------------+--------------------+-----------+---------------+-------+
|      JOUR|CODE_STIF_TRNS|CODE_STIF_RES|CODE_STIF_ARRET|       LIBELLE_ARRET|ID_REFA_LDA|CATEGORIE_TITRE|NB_VALD|
+----------+--------------+-------------+---------------+--------------------+-----------+---------------+-------+
|2016-01-05|           100|          110|            762|SAINT-DENIS-PORTE...|      72285|      AMETHYSTE|    264|
|2016-01-05|           100|          110|            766|       SAINT-GEORGES|      71366|            TST|    103|
|2016-01-05|           100|          110|            768|       SAINT-JACQUES|      71041|            TST|    415|
|2016-01-05|           100|          110|            769|        SAINT-LAZARE|      71370|      IMAGINE R|  15406|
|2016-01-05|           100|          110|            777|SAINT-PAUL (LE MA...|      71222|    AUTRE TITRE|     88|
+----------+--------------+-------------+---------------+--------------------+--

## 1. Total, min, max, écart type de validations sur l'ensemble du réseau

In [47]:
dfValidation.printSchema()

root
 |-- JOUR: date (nullable = true)
 |-- CODE_STIF_TRNS: string (nullable = true)
 |-- CODE_STIF_RES: string (nullable = true)
 |-- CODE_STIF_ARRET: string (nullable = true)
 |-- LIBELLE_ARRET: string (nullable = true)
 |-- ID_REFA_LDA: string (nullable = true)
 |-- CATEGORIE_TITRE: string (nullable = true)
 |-- NB_VALD: integer (nullable = true)



In [49]:
// Register the DataFrame as a global temporary view
dfValidation.createOrReplaceTempView("validation")
spark.sql("desc validation").show()


+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|           JOUR|     date|   null|
| CODE_STIF_TRNS|   string|   null|
|  CODE_STIF_RES|   string|   null|
|CODE_STIF_ARRET|   string|   null|
|  LIBELLE_ARRET|   string|   null|
|    ID_REFA_LDA|   string|   null|
|CATEGORIE_TITRE|   string|   null|
|        NB_VALD|      int|   null|
+---------------+---------+-------+



In [50]:
spark.sql("select MAX(NB_VALD) from validation").collect()


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 38.0 failed 10 times, most recent failure: Lost task 0.9 in stage 38.0 (TID 240, yp-spark-dal09-env5-0049, executor b960e61c-e829-4b0a-bdf5-48d516cf0e96): java.lang.NumberFormatException: For input string: "Moins de 5"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:76)
	at java.lang.Integer.parseInt(Integer.java:592)
	at java.lang.Integer.parseInt(Integer.java:627)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:272)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at org.apache.spark.sql.execution.datasources.csv.CSVTypeCast$.castTo(CSVInferSchema.scala:250)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:125)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:94)
	at org.apache.spark.sql.execution.data

In [ ]:
 val rdd = sc.parallelize(
      List( (2012,"Tesla","S"), (1997,"Ford","E350"), (2015,"Chevy","Volt"))
  )
  val sqlContext = new SQLContext(sc)

  // this is used to implicitly convert an RDD to a DataFrame.
  import sqlContext.implicits._

  val dataframe = rdd.toDF()

  dataframe.foreach(println)

 dataframe.map(row => {
    val row1 = row.getAs[String](1)
    val make = if (row1.toLowerCase == "tesla") "S" else row1
    Row(row(0),make,row(2))
  }).collect().foreach(println)

//[2012,S,S]
//[1997,Ford,E350]
//[2015,Chevy,Volt]

# "Moins de 5" to 0